In [5]:
from telemetry_class import Telemetry
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
import json
from datetime import datetime
from json import dumps
keys=[]
results=[]
summary_list=[]
start_time=datetime.now()



In [6]:
spark=SparkSession.builder.appName("push-to-kafka").config('spark.sql.debug.maxToStringFields', 1000).getOrCreate()

In [8]:
telemetry=spark.read.json('../../data/2021-01-01-1-1609483435390.json.gz', encoding='utf-8')

In [9]:
# telemetry.printSchema()

root
 |-- @timestamp: string (nullable = true)
 |-- @version: string (nullable = true)
 |-- actor: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- context: struct (nullable = true)
 |    |-- cdata: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |-- channel: string (nullable = true)
 |    |-- did: string (nullable = true)
 |    |-- env: string (nullable = true)
 |    |-- pdata: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- pid: string (nullable = true)
 |    |    |-- ver: string (nullable = true)
 |    |-- rollup: struct (nullable = true)
 |    |    |-- l1: string (nullable = true)
 |    |    |-- l2: string (nullable = true)
 |    |    |-- l3: string (nullable = true)
 |    |-- sid: string (nullable = true)
 |    |-- uid: string (nullable = true)
 |-- eda

In [17]:
telemetry.count()

930000

In [61]:
metrics=telemetry.limit(5)

In [77]:
 print(metrics)

DataFrame[@timestamp: string, @version: string, actor: struct<id:string,type:string>, context: struct<cdata:array<struct<id:string,type:string>>,channel:string,did:string,env:string,pdata:struct<id:string,pid:string,ver:string>,rollup:struct<l1:string,l2:string,l3:string>,sid:string,uid:string>, edata: struct<commentid:string,comments:string,commenttxt:string,data:string,dir:string,dspec:struct<camera:string,cpu:string,edisk:double,id:string,idisk:double,make:string,os:string,scrn:double,sims:double,webview:string>,duration:double,extra:struct<audience:array<string>,board:array<string>,contactType:string,filters:struct<audience:array<string>,board:array<string>,channel:array<string>,gradeLevel:array<string>,key:string,medium:array<string>,selectedTab:string,subject:array<string>>,form:struct<id:string,identifier:string,name:string,type:string,value:string>,gradeLevel:array<string>,isError:string,key:string,medium:array<string>,pos:array<string>,query:string,section:string,stageProgress

In [65]:
metrics.write.json('./filter/')


DataFrame[@timestamp: string, @version: string, actor: struct<id:string,type:string>, context: struct<cdata:array<struct<id:string,type:string>>,channel:string,did:string,env:string,pdata:struct<id:string,pid:string,ver:string>,rollup:struct<l1:string,l2:string,l3:string>,sid:string,uid:string>, edata: struct<commentid:string,comments:string,commenttxt:string,data:string,dir:string,dspec:struct<camera:string,cpu:string,edisk:double,id:string,idisk:double,make:string,os:string,scrn:double,sims:double,webview:string>,duration:double,extra:struct<audience:array<string>,board:array<string>,contactType:string,filters:struct<audience:array<string>,board:array<string>,channel:array<string>,gradeLevel:array<string>,key:string,medium:array<string>,selectedTab:string,subject:array<string>>,form:struct<id:string,identifier:string,name:string,type:string,value:string>,gradeLevel:array<string>,isError:string,key:string,medium:array<string>,pos:array<string>,query:string,section:string,stageProgress

In [71]:
metricsjson=json.dumps(json.load(open('./filter/data.json')))

In [74]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], value_serializer=lambda x: x.encode('utf-8'))
producer.send('sb_telemetry', metricsjson)
print("events pushed into kafka successfully")

events pushed into kafka successfully


In [75]:
print(metricsjson)

[{"@timestamp": "2021-01-01T06:06:04.240Z", "actor": {"id": "939f2ae5-d9cc-47c7-a961-3dce577211fa", "type": "User"}, "context": {"cdata": [{"id": "607ca980-4bf7-11eb-9aa3-8bad8b0879fc", "type": "API"}, {"id": "SearchResult", "type": "Section"}, {"id": "", "type": "CourseBatch"}, {"id": "do_31317601015225548814303", "type": "RootId"}, {"id": "c0df5d06-2fbd-4adc-9036-040457beb5ff", "type": "UserSession"}], "channel": "01248978648941363234", "did": "29bbe3e293f99602e14ee8a35b549b8c31584267", "env": "app", "pdata": {"id": "prod.diksha.app", "pid": "sunbird.app", "ver": "3.4.578"}, "rollup": {"l1": "01248978648941363234"}, "sid": "c0df5d06-2fbd-4adc-9036-040457beb5ff"}, "edata": {"extra": {"pos": []}, "id": "ImportContent", "pageid": "ImportContent", "subtype": "online", "type": "OTHER"}, "eid": "INTERACT", "ets": 1609481147307.0, "flags": {"ex_processed": true, "pp_duplicate_skipped": true, "pp_validation_processed": true}, "mid": "2fab96ab-60dd-472b-bb33-e17f86a105a5", "object": {"id": "d